In [1]:
import gspread
import pandas as pd
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import warnings
from icalendar import Calendar
from icalendar import Event
from datetime import datetime, date, time
import uuid  # Para criar IDs únicos para cada evento
from pandas import ExcelWriter

# Suprimir avisos de depreciação do pandas, comuns em notebooks
warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Constantes ---
# Chave da sua planilha (extraída da URL)
SHEET_KEY = '1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY'
# Arquivo de credenciais que você forneceu
CREDS_FILE = 'credentials.json'
# Arquivo para armazenar o token de acesso após a primeira autorização
TOKEN_FILE = 'token.json'
# Escopo de permissão (apenas leitura)
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

In [2]:
def authenticate_google_sheets():
    """
    Lida com o fluxo de autenticação OAuth 2.0 para um app "instalado".
    Usa 'credentials.json' para obter um 'token.json'.
    """
    creds = None
    # O 'token.json' armazena os tokens de acesso e atualização do usuário.
    # Ele é criado automaticamente no primeiro login bem-sucedido.
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    
    # Se não houver credenciais (válidas), permite que o usuário faça login.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Renovando token de acesso...")
            creds.refresh(Request())
        else:
            print("Nenhum token válido encontrado. Iniciando fluxo de autenticação...")
            # 'InstalledAppFlow' é o método correto para o seu tipo de credentials.json
            flow = InstalledAppFlow.from_client_secrets_file(CREDS_FILE, SCOPES)
            
            # run_local_server é o mais fácil para Jupyter Notebook local.
            # Ele abrirá uma aba no seu navegador para autorização.
            creds = flow.run_local_server(port=0)
            print("Autenticação concluída.")
            
        # Salva as credenciais para as próximas execuções
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
            print(f"Token salvo em {TOKEN_FILE}")
            
    return gspread.authorize(creds)

In [3]:
def baixar_dados_google_sheet(tab_name, sheet_key):
    """
    Função principal para ler e processar os dados da planilha.
    
    Lê a aba 'CBL004', assume que os cabeçalhos estão na linha 3
    e os dados começam na linha 4. Retorna um DataFrame
    do Pandas apenas com as colunas A, B, e F.
    """
    try:
        # 1. Autenticar
        gc = authenticate_google_sheets()
        
        # 2. Abrir a planilha e a aba
        print(f"Abrindo planilha (ID: {sheet_key})...")
        sh = gc.open_by_key(sheet_key)
        worksheet = sh.worksheet(tab_name)
        
        # 3. Ler todos os valores da aba
        print(f"Lendo todos os dados da aba '{tab_name}'...")
        all_values = worksheet.get_all_values()
        
        # 4. Processar com Pandas
        # O pedido foi "starting from row 4".
        # Vamos assumir que a linha 3 (índice 2) contém os cabeçalhos
        # E a linha 4 (índice 3) é o início dos dados.
        if len(all_values) < 4:
            print(f"Aviso: A aba '{tab_name}' tem menos de 4 linhas. Retornando DataFrame vazio.")
            return pd.DataFrame()

        headers = all_values[2]  # Linha 3 (índice 2) como cabeçalhos
        data = all_values[3:]    # Dados da Linha 4 (índice 3) em diante
        
        df = pd.DataFrame(data, columns=headers)
            # Salva o DataFrame em uma planilha Excel
        try:
            planilha_output_path = 'dados_google_sheet.xlsx'
            #print(f"Salvando planilha em '{planilha_output_path}'...")
            #df.to_excel(planilha_output_path, sheet_name=tab_name, index=False, engine='openpyxl')
            with ExcelWriter(planilha_output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=tab_name, index=False)
                print(f"Planilha '{planilha_output_path}' - '{tab_name}'salva com sucesso!")
        except Exception as e_save:
            print(f"ERRO ao salvar a planilha Excel '{planilha_output_path}' - '{tab_name}': {e_save}")

        
        # 5. Filtrar para colunas A, B, F
        # As colunas A, B, F estão nos índices 0, 1, 5 da lista de cabeçalhos
        try:
            col_a_name = headers[0]
            col_b_name = headers[1]
            col_f_name = headers[5]
            
            # Seleciona apenas as colunas desejadas pelos nomes
            df_filtered = df[[col_a_name, col_b_name, col_f_name]]
            
            print("Processamento concluído com sucesso.")
            return df_filtered
            
        except IndexError:
            print(f"Erro: A planilha não parece ter 6 colunas (A-F) com cabeçalhos na linha 3.")
            print(f"Cabeçalhos encontrados ({len(headers)}): {headers}")
            return pd.DataFrame() # Retorna DF vazio

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Erro: Planilha com a chave {SHEET_KEY} não encontrada.")
        print("Verifique o link e se sua conta tem permissão de leitura.")
        return None
    except gspread.exceptions.WorksheetNotFound:
        print(f"Erro: Aba '{tab_name}' não encontrada na planilha.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

In [4]:
'''import pandas as pd
import gspread # Certifique-se de que está importado
# Presume que authenticate_google_sheets e SHEET_KEY estão definidos

def baixar_dados_google_sheet_com_notas(tab_name, sheet_key):
    """
    Função para ler dados e NOTAS da planilha Google Sheets, garantindo alinhamento.
    
    Lê a aba especificada, assume cabeçalhos na linha 3, dados na linha 4.
    Retorna um DataFrame do Pandas com as colunas A, B, F e suas respectivas notas.
    """
    try:
        # 1. Autenticar
        gc = authenticate_google_sheets()
        
        # 2. Abrir a planilha e a aba
        print(f"Abrindo planilha (ID: {sheet_key})...")
        sh = gc.open_by_key(sheet_key)
        worksheet = sh.worksheet(tab_name)
        
        # 3. Ler todos os valores da aba
        print(f"Lendo todos os dados da aba '{tab_name}'...")
        all_values = worksheet.get_all_values()
        
        # 4. Validar se há dados suficientes (cabeçalho + pelo menos 1 linha de dados)
        if len(all_values) < 4:
            print(f"Aviso: A aba '{tab_name}' tem menos de 4 linhas. Retornando DataFrame vazio.")
            return pd.DataFrame()

        # Extrai cabeçalhos (linha 3, índice 2)
        headers = all_values[2]
        # Dados começam na linha 4 (índice 3)
        data_values = all_values[3:]
        num_data_rows = len(data_values)
        
        # Define os nomes das colunas de interesse
        try:
            col_a_name = headers[0]
            col_b_name = headers[1]
            col_f_name = headers[5]
        except IndexError:
            print(f"Erro: A planilha não parece ter 6 colunas (A-F) com cabeçalhos na linha 3.")
            print(f"Cabeçalhos encontrados ({len(headers)}): {headers}")
            return pd.DataFrame()

        # 5. Ler as Notas para o intervalo correspondente aos dados
        last_data_row_num_sheet = num_data_rows + 3 # Número da última linha na planilha
        notes_range_label = f"A4:F{last_data_row_num_sheet}" 
        raw_notes = []
        try:
            print(f"Lendo notas do intervalo '{notes_range_label}'...")
            raw_notes = worksheet.get_notes(notes_range_label)
            print(f"Leitura das notas concluída ({len(raw_notes)} linhas de notas retornadas).")
        except Exception as e_notes:
            print(f"Aviso: Não foi possível ler as notas da planilha: {e_notes}. As colunas de nota ficarão vazias.")
            # Cria uma estrutura vazia com as dimensões corretas se a leitura falhar
            raw_notes = [[''] * 6 for _ in range(num_data_rows)]

        # --- 6. Construir a lista de dados combinados (valores + notas) ---
        combined_data = []
        for i in range(num_data_rows):
            row_values = data_values[i]
            
            # Pega as notas correspondentes, preenchendo com '' se a nota não existir
            # (raw_notes pode ser mais curta que data_values se houver linhas finais sem notas)
            row_notes = raw_notes[i] if i < len(raw_notes) else [''] * 6 
            
            # Garante que a linha de notas tenha pelo menos 6 colunas
            padded_row_notes = (row_notes + [''] * 6)[:6] 

            # Cria o dicionário para esta linha
            row_dict = {
                col_a_name: row_values[0] if len(row_values) > 0 else '',
                col_b_name: row_values[1] if len(row_values) > 1 else '',
                col_f_name: row_values[5] if len(row_values) > 5 else '',
                'Nota_A': padded_row_notes[0],
                'Nota_B': padded_row_notes[1],
                'Nota_F': padded_row_notes[5]
            }
            combined_data.append(row_dict)

        # 7. Criar o DataFrame Final a partir da lista de dicionários
        df_final = pd.DataFrame(combined_data)
        
        print("Processamento concluído com sucesso.")
        return df_final

    # Blocos except (sem alteração)
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Erro: Planilha com a chave {sheet_key} não encontrada.")
        print("Verifique o link e se sua conta tem permissão de leitura.")
        return None
    except gspread.exceptions.WorksheetNotFound:
        print(f"Erro: Aba '{tab_name}' não encontrada na planilha.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

# --- Exemplo de como chamar a nova função ---
# (Assumindo que authenticate_google_sheets e SHEET_KEY estão definidos)
#df_com_notas = baixar_dados_google_sheet_com_notas('CBL004', SHEET_KEY)
# if df_com_notas is not None:
#     # display(df_com_notas.head())
#     # Mostra as linhas do índice 355 até o 365 (o índice 366 é exclusivo)
#display(df_com_notas.iloc[355:366]) 
#     #display(df_com_notas.tail())'''

'import pandas as pd\nimport gspread # Certifique-se de que está importado\n# Presume que authenticate_google_sheets e SHEET_KEY estão definidos\n\ndef baixar_dados_google_sheet_com_notas(tab_name, sheet_key):\n    """\n    Função para ler dados e NOTAS da planilha Google Sheets, garantindo alinhamento.\n    \n    Lê a aba especificada, assume cabeçalhos na linha 3, dados na linha 4.\n    Retorna um DataFrame do Pandas com as colunas A, B, F e suas respectivas notas.\n    """\n    try:\n        # 1. Autenticar\n        gc = authenticate_google_sheets()\n        \n        # 2. Abrir a planilha e a aba\n        print(f"Abrindo planilha (ID: {sheet_key})...")\n        sh = gc.open_by_key(sheet_key)\n        worksheet = sh.worksheet(tab_name)\n        \n        # 3. Ler todos os valores da aba\n        print(f"Lendo todos os dados da aba \'{tab_name}\'...")\n        all_values = worksheet.get_all_values()\n        \n        # 4. Validar se há dados suficientes (cabeçalho + pelo menos 1 lin

In [5]:
def formatar_dataframe_reservas(df_original):
    """
    Converte um DataFrame com colunas 'Início', 'Fim' e 'Origem' para o 
    formato de Gantt/Timeline ('Start', 'End', 'Summary') com os Dtypes corretos.

    Assume que 'Início' é sempre às 16:00 (check-in) e 'Fim' é sempre 
    às 11:00 (check-out).
    """
    
    # 1. Fazer uma cópia para evitar o SettingWithCopyWarning
    df = df_original.copy()

    # 2. Mapa para converter meses em português (pt-br) para números
    meses_map = {
        'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 
        'mai': '05', 'jun': '06', 'jul': '07', 'ago': '08', 
        'set': '09', 'out': '10', 'nov': '11', 'dez': '12'
    }

    # 3. Renomear e capitalizar a coluna 'Origem'
    df = df.rename(columns={'Origem': 'Summary'})
    df['Summary'] = df['Summary'].str.capitalize()

    # 4. Processar colunas de data ('Início' e 'Fim')
    colunas_data = ['Início', 'Fim']
    novos_nomes = ['Start', 'End']
    
    for i, col_nome in enumerate(colunas_data):
        
        # Etapa A: Remover o dia da semana E os espaços em branco
        datas_limpas = df[col_nome].str.rsplit('-', n=1).str[0]
        datas_limpas = datas_limpas.str.strip() 
        
        # Etapa B: Substituir os meses em português por números
        temp_series = datas_limpas.str.lower()
        for pt_mes, num_mes in meses_map.items():
            temp_series = temp_series.str.replace(pt_mes, num_mes)
        
        # Etapa C: Converter para datetime (ainda ficará 00:00:00)
        novo_nome_col = novos_nomes[i]
        df[novo_nome_col] = pd.to_datetime(temp_series, format='%d-%m.%y', errors='coerce')

    
    # --- MUDANÇA PRINCIPAL: Adicionar horas específicas ---
    # (Verifica se a coluna existe e não é NaT antes de somar)
    
    # 5. Adicionar 16:00 (check-in) à coluna 'Start'
    if 'Start' in df.columns:
        start_valid = df['Start'].notna()
        df.loc[start_valid, 'Start'] = df.loc[start_valid, 'Start'] + pd.to_timedelta('16 hours')

    # 6. Adicionar 11:00 (check-out) à coluna 'End'
    if 'End' in df.columns:
        end_valid = df['End'].notna()
        df.loc[end_valid, 'End'] = df.loc[end_valid, 'End'] + pd.to_timedelta('11 hours')

    # ----------------------------------------------------

    # 7. Remover as colunas originais 'Início' e 'Fim'
    # Usamos errors='ignore' caso elas já tenham sido substituídas
    df = df.drop(columns=['Início', 'Fim'], errors='ignore')
    
    # 8. Manter apenas as colunas finais
    colunas_finais = ['Start', 'End', 'Summary']
    # Filtra para garantir que apenas colunas existentes sejam selecionadas
    colunas_presentes = [col for col in colunas_finais if col in df.columns]
    df = df[colunas_presentes]

    # 9. Remover linhas onde a data falhou (virou NaT)
    linhas_com_erro = df['Start'].isna() | df['End'].isna()
    if linhas_com_erro.any():
        print(f"Aviso: {linhas_com_erro.sum()} linha(s) não puderam ser convertidas e foram removidas.")
        df = df.dropna(subset=['Start', 'End'])
    
    return df

In [6]:
def save_dataframe_to_ical(df, 
                      output_filename, 
                      start_col='Start', 
                      end_col='End', 
                      summary_col='Summary', 
                      description_col=None, 
                      location_col=None):
    """
    Converte um DataFrame do Pandas em um arquivo .ics (iCalendar).

    Args:
        df (pd.DataFrame): O DataFrame contendo os dados do evento.
        output_filename (str): O nome do arquivo .ics a ser criado (ex: 'meu_calendario.ics').
        start_col (str): O nome da coluna com o datetime de início (default: 'Start').
        end_col (str): O nome da coluna com o datetime de fim (default: 'End').
        summary_col (str): O nome da coluna com o título/resumo do evento (default: 'Summary').
        description_col (str, opcional): Coluna para a descrição do evento.
        location_col (str, opcional): Coluna para a localização do evento.
    """
    
    # 1. Validar colunas obrigatórias
    colunas_obrigatorias = [start_col, end_col, summary_col]
    for col in colunas_obrigatorias:
        if col not in df.columns:
            print(f"Erro: A coluna obrigatória '{col}' não foi encontrada no DataFrame.")
            return

    # 2. Criar o objeto Calendário principal
    cal = Calendar()
    cal.add('prodid', '-//Gerador de Calendário a partir de Pandas//exemplo.com//')
    cal.add('version', '2.0')

    # 3. Iterar sobre o DataFrame e criar Eventos
    for index, row in df.iterrows():
        
        # Pular linhas com dados essenciais faltando
        if pd.isna(row[start_col]) or pd.isna(row[end_col]) or pd.isna(row[summary_col]):
            print(f"Aviso: Pulando linha {index} por conter dados Nulos em colunas essenciais.")
            continue
            
        event = Event()
        
        # --- Dados Obrigatórios ---
        event.add('summary', row[summary_col])
        event.add('dtstart', row[start_col])  # A biblioteca aceita Timestamps do Pandas
        event.add('dtend', row[end_col])
        
        # --- Metadados do Evento ---
        event.add('dtstamp', datetime.now()) # Data de criação do evento
        event.add('uid', str(uuid.uuid4()))      # ID Único
        
        # --- Dados Opcionais ---
        if description_col and description_col in df.columns and pd.notna(row[description_col]):
            event.add('description', row[description_col])
            
        if location_col and location_col in df.columns and pd.notna(row[location_col]):
            event.add('location', row[location_col])

        # 4. Adicionar o evento ao calendário
        cal.add_component(event)

    # 5. Escrever o calendário no arquivo
    try:
        with open(output_filename, 'wb') as f:
            # .to_ical() retorna bytes, por isso abrimos o arquivo com 'wb' (write bytes)
            f.write(cal.to_ical())
        #print(f"Calendário salvo com sucesso em '{output_filename}'")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

In [7]:
# Mapeie o 'ID' (usado nos nomes dos arquivos) para o 'Nome da Aba' no Google Sheets
apartment_map = {
    'c108': 'SM-C108',
    'd014': 'SM-D014',
    'cbl004': 'CBL004',
    'ap101': 'AP-101',
    'ap201': 'AP-201',
    'f216': 'SM-F216',
    # Adicione mais apartamentos aqui conforme necessário
}

print("Iniciando o processamento dos calendários do Google Sheets...")

# Itera sobre cada item (chave e valor) no dicionário
for apt, tab_name in apartment_map.items():
    
    print(f"\n--- Processando Apartamento: {apt.upper()} (Aba: {tab_name}) ---")
    
    # Define o nome do arquivo de saída .ics dinamicamente
    file_google_calendar = f'calendars/{apt}_google.ics'
    
    try:
        # 1. Baixa os dados da aba específica
        df_reservas_google_sheet = baixar_dados_google_sheet(tab_name=tab_name, sheet_key=SHEET_KEY)
        
        if df_reservas_google_sheet is None or df_reservas_google_sheet.empty:
            print(f"Aviso: Não foram encontrados dados para a aba '{tab_name}'. Pulando.")
            continue # Pula para o próximo apartamento

        # 2. Formata o DataFrame
        df_reservas_google_sheet = formatar_dataframe_reservas(df_reservas_google_sheet)

        # 3. Salva o DataFrame como um arquivo iCal
        save_dataframe_to_ical(df_reservas_google_sheet, output_filename=file_google_calendar)
        
        print(f"Calendário Google salvo com sucesso em: {file_google_calendar}")

    except Exception as e:
        print(f"ERRO: Falha ao processar o apartamento {apt.upper()}: {e}")

print("\n--- Processamento de todos os apartamentos concluído! ---")

Iniciando o processamento dos calendários do Google Sheets...

--- Processando Apartamento: C108 (Aba: SM-C108) ---
Renovando token de acesso...
Token salvo em token.json
Abrindo planilha (ID: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY)...
Lendo todos os dados da aba 'SM-C108'...
ERRO ao salvar a planilha Excel 'dados_google_sheet.xlsx' - 'SM-C108': [Errno 2] No such file or directory: 'dados_google_sheet.xlsx'
Processamento concluído com sucesso.
Aviso: 41 linha(s) não puderam ser convertidas e foram removidas.
Calendário Google salvo com sucesso em: calendars/c108_google.ics

--- Processando Apartamento: D014 (Aba: SM-D014) ---
Abrindo planilha (ID: 1FqgTQAGebxvHUdVXI471HpAaXeXyCFdFWur7Pck0hLY)...
Lendo todos os dados da aba 'SM-D014'...
ERRO ao salvar a planilha Excel 'dados_google_sheet.xlsx' - 'SM-D014': [Errno 2] No such file or directory: 'dados_google_sheet.xlsx'
Processamento concluído com sucesso.
Calendário Google salvo com sucesso em: calendars/d014_google.ics

--- Proces